In [8]:
import os

In [9]:
%pwd   ## to check current directory

'c:\\Projects\\Text-Summarization\\research'

In [10]:
os.chdir("../")  ## to change directory to parent directory

In [11]:
%pwd

'c:\\Projects\\Text-Summarization'

In [2]:
## This is notebook experimental code, not intended for production use.

In [12]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [13]:
from TextSummarization.constants import *
from TextSummarization.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=  CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from TextSummarization.logging import logger
from TextSummarization.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {filename}!with following info: \n{headers}")
        else:
            logger.info(f"file already exists with size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
## Pipeline creation

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-04 16:31:16,839] INFO: common: Successfully loaded yaml file: config\config.yaml]
[2024-08-04 16:31:16,842] INFO: common: Successfully loaded yaml file: params.yaml]
[2024-08-04 16:31:16,844] INFO: common: Created directory: artifacts]
[2024-08-04 16:31:16,848] INFO: common: Created directory: artifacts/data_ingestion]
[2024-08-04 16:31:39,143] INFO: 4257573218: Downloaded artifacts/data_ingestion/data.zip!with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4485:1860EE:13A272:1B2793:66AF5F7C
Accept-Ranges: bytes
Date: Sun, 04 Aug 2024 11:01:18 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4724-BOM
X-Cache: MISS
X-

In [2]:
## to check the current directory
import os
print(os.getcwd())   ## note it should be in TextSummarization directory.

c:\Projects\Text-Summarization\research
